<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/DAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перейити в гугл диск и загрузить в корень диска файл

https://drive.google.com/drive/

In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла: ") or "input.mp4"

In [ ]:
#@title ##**Проверка GPU** { display-mode: "form" }
!nvidia-smi

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Установка библиотек** { display-mode: "form" }
!pip install torch
!pip install torchvision
!pip install scipy
!pip install Pillow
!pip install requests
!pip install apex
!pip install pyramid
!pip install zope.interface
!pip install zope
import torch
print(torch.__version__)

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
!git clone https://github.com/baowenbo/DAIN /content/DAIN
%cd /content
!rm -rf DAIN
!git clone --depth 1 https://github.com/AlphaGit/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

In [6]:
#@title ##**Замена кода** { display-mode: "form" }
import os
import re

def process_files(root_dir):
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file == 'setup.py':
                filepath = os.path.join(subdir, file)
                with open(filepath, 'r') as f:
                    content = f.read()

                new_content = re.sub(
                    r'^from compiler_args import nvcc_args, cxx_args',
                    ''
                    r'nvcc_args = [\n'
                    r"    '-gencode', 'arch=compute_61,code=sm_61',\n"
                    r"    '-gencode', 'arch=compute_75,code=sm_75',\n"
                    r"    '-gencode', 'arch=compute_86,code=sm_86',\n"
                    r"    '--expt-relaxed-constexpr'\n"
                    r']\n'
                    r'cxx_args = [\n'
                    r"    '-std=c++14',\n"
                    r"    '-O2',\n"
                    r"    '-Wall',\n"
                    r"    '-fPIC'\n"
                    r']\n',
                    content,
                    flags=re.MULTILINE
                )

                with open(filepath, 'w') as f:
                    f.write(new_content)


process_files('/content/DAIN')

# открываем файл
with open('/content/DAIN/PWCNet/correlation_package_pytorch1_0/correlation.py', 'r') as f:
    contents = f.read()

new_text = '''
import torch
from torch.nn.modules.module import Module
from torch.autograd import Function
import correlation_cuda

class CorrelationFunction(Function):
    @staticmethod
    def forward(ctx, input1, input2, pad_size, kernel_size, max_displacement, stride1, stride2, corr_multiply):
        ctx.save_for_backward(input1, input2)
        ctx.pad_size = pad_size
        ctx.kernel_size = kernel_size
        ctx.max_displacement = max_displacement
        ctx.stride1 = stride1
        ctx.stride2 = stride2
        ctx.corr_multiply = corr_multiply

        with torch.cuda.device_of(input1):
            rbot1 = input1.new_empty((input1.size(0), (max_displacement // stride2 * 2 + 1)**2, input1.size(2) + 2 * pad_size, input1.size(3) + 2 * pad_size))
            rbot2 = input2.new_empty((input2.size(0), (max_displacement // stride2 * 2 + 1)**2, input2.size(2) + 2 * pad_size, input2.size(3) + 2 * pad_size))
            output = input1.new_empty((input1.size(0), (max_displacement // stride2 * 2 + 1)**2, int((input1.size(2) + 2 * pad_size - kernel_size) / stride1 + 1), int((input1.size(3) + 2 * pad_size - kernel_size) / stride1 + 1)))

            correlation_cuda.forward(input1, input2, rbot1, rbot2, output, pad_size, kernel_size, max_displacement, stride1, stride2, corr_multiply)

        return output

    @staticmethod
    def backward(ctx, grad_output):
        input1, input2 = ctx.saved_tensors
        pad_size = ctx.pad_size
        kernel_size = ctx.kernel_size
        max_displacement = ctx.max_displacement
        stride1 = ctx.stride1
        stride2 = ctx.stride2
        corr_multiply = ctx.corr_multiply

        with torch.cuda.device_of(input1):
            grad_input1 = input1.new_empty(input1.size())
            grad_input2 = input2.new_empty(input2.size())

            correlation_cuda.backward(input1, input2, grad_output.contiguous(), grad_input1, grad_input2, ctx.buffer1, ctx.buffer2, ctx.buffer3, pad_size, kernel_size, max_displacement, stride1, stride2, corr_multiply)

        return grad_input1, grad_input2, None, None, None, None, None

class Correlation(Module):
    def __init__(self, pad_size=0, kernel_size=0, max_displacement=0, stride1=1, stride2=2, corr_multiply=1):
        super(Correlation, self).__init__()
        self.pad_size = pad_size
        self.kernel_size = kernel_size
        self.max_displacement = max_displacement
        self.stride1 = stride1
        self.stride2 = stride2
        self.corr_multiply = corr_multiply

    def forward(self, input1, input2):
        return CorrelationFunction.apply(input1, input2, self.pad_size, self.kernel_size, self.max_displacement, self.stride1, self.stride2, self.corr_multiply)
'''

with open('/content/DAIN/PWCNet/correlation_package_pytorch1_0/correlation.py', 'w') as f:
    f.write(new_text)

# Новая строка
new_string = "https://raw.githubusercontent.com/googlefonts/roboto/main/src/hinted/Roboto-Bold.ttf"

# Открываем файл локально и читаем его содержимое
filename = '/content/DAIN/colab_interpolate.py'
with open(filename, 'r') as f:
    script_content = f.read()

# Изменяем путь к модели в строке скрипта
new_script_content = re.sub(r"https://github.com/googlefonts/roboto/blob/master/src/hinted/Roboto-Bold.ttf\?raw=true", new_string, script_content)

# Записываем измененный скрипт в файл
with open(filename, 'w') as f:
    f.write(new_script_content)


In [7]:
#@title ##**Инициализация ☕️** { display-mode: "form" }

# Building DAIN.
%%capture
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")
# Wait again. ~20 minutes.
# Building DAIN PyTorch correlation package.

%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

In [ ]:
#@title ##**Загрузка модели** { display-mode: "form" }
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [9]:
#@title ##**Установка imagemagick** { display-mode: "form" }
%%capture
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")

In [27]:
#@title ##**Конфигурация** { display-mode: "form" }
path = '/mnt/gdrive/MyDrive/'

INPUT_FILEPATH =  os.path.join(path, file_name)

OUTPUT_FILE_PATH = "/content/DAIN/output.mp4"

TARGET_FPS =  80#@param{type:"number"}

FRAME_INPUT_DIR = '/mnt/gdrive/MyDrive/dain_input'

FRAME_OUTPUT_DIR = '/mnt/gdrive/MyDrive/dain_output'

SEAMLESS = False #@param{type:"boolean"}

RESIZE_HOTFIX = False #@param{type:"boolean"}

AUTO_REMOVE = False

In [28]:
#@title ##**Создать папки для изображений** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт
from google.colab import files
import shutil
import glob
import os

input_image_folder = '/mnt/gdrive/MyDrive/dain_input'
output_image_folder = '/mnt/gdrive/MyDrive/dain_output'

if not os.path.exists(input_image_folder):
    os.makedirs(input_image_folder)


if not os.path.exists(output_image_folder):
    os.makedirs(output_image_folder)


In [38]:
#@title ##**Очистить входящую папку** { display-mode: "form" }
if os.path.isdir(input_image_folder):
    shutil.rmtree(input_image_folder)

os.makedirs(input_image_folder)

In [39]:
#@title ##**Очистить исходящую папку** { display-mode: "form" }
if os.path.isdir(output_image_folder):
    shutil.rmtree(output_image_folder)

os.makedirs(output_image_folder)

In [ ]:
#@title ##**Создать секвенцию** { display-mode: "form" }
# Detecting FPS of input file.
#%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /contIN/ent/DA
#%shell yes | cp -f /content/{INPUT_FILEPATH} 

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(INPUT_FILEPATH)

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")
# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '{INPUT_FILEPATH}' -vf "setpts=1.1*PTS" '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
from IPython.display import clear_output
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))
#%shell mkdir -p '{FRAME_OUTPUT_DIR}'

In [35]:
#@title ##**Создать лог файл** { display-mode: "form" }

dir_path = '/content/DAIN'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

file_path = os.path.join(dir_path, 'scene_frames.log')
with open(file_path, 'w') as f:
    f.write('')

In [ ]:
#@title ##**Запуск** { display-mode: "form" }
#%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN
print(fps)

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'


In [43]:
#@title ##**Переименовать секвенцию** { display-mode: "form" }
import glob
import os
import time

png_files = sorted(glob.glob('/mnt/gdrive/MyDrive/dain_output/*.png'))
png_files.sort()

for i, file in enumerate(png_files):
    new_filename = '{:05d}.png'.format(i+1)
    success = False
    while not success:
        try:
            os.rename(file, os.path.join(os.path.dirname(file), new_filename))
            success = True
        except Exception as e:
            print(f"Error renaming {file} to {new_filename}: {e}")
            time.sleep(1)  # ждем 1 секунду перед повторной попыткой

In [ ]:
#@title ##**Подогнать под оригинальный размер** { display-mode: "form" }
# Finding DAIN Frames, upscaling and cropping to match original(optional)
import numpy as np
import os
import cv2
%cd {FRAME_OUTPUT_DIR}
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)

%cd /content/DAIN

In [ ]:
#@title ##**Сборка видео** { display-mode: "form" }
import cv2
import os
from tqdm.notebook import tqdm

output_file = "/mnt/gdrive/MyDrive/output.mp4"
path = '/mnt/gdrive/MyDrive/'

full_path = os.path.join(path, file_name)

cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

img_files = os.listdir(output_image_folder)
img_files.sort()

frame = cv2.imread(os.path.join(output_image_folder, img_files[0]))
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, TARGET_FPS, (width, height))

for img_file in tqdm(img_files, desc='Сборка видео'):
    img_path = os.path.join(output_image_folder, img_file)
    frame = cv2.imread(img_path)

    writer.write(frame)

writer.release()
print('Сборка завершена')

In [ ]:
#@title ##**Добавление звука** { display-mode: "form" }
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip

video = VideoFileClip(output_file)

audio = AudioFileClip(full_path)

video = video.set_audio(audio)

video.write_videofile('/mnt/gdrive/MyDrive/output_with_audio.mp4')

In [ ]:
#@title ##**Скачать файл (опционально, быстерее можно скачать в гугл диске)** { display-mode: "form" }
from google.colab import files
files.download('/mnt/gdrive/MyDrive/output_with_audio.mp4')